In [1]:
import os
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt

In [3]:
folders = os.listdir(r"/Users/Cate/Desktop/DSBA/M2/MLNS/project")

In [81]:
#we want to open any folder and save just 0.txt
df_list = []
failed = []
for i in folders:
    path = f"{i}/0.txt"
    try:
        df = pd.read_csv(path, sep = '\t', header= None)
    except:
        print(f"Error importing file:{i}")
        failed.append(i)
        continue
    df_list.append(df)

Error importing file:0302
Error importing file:0314
Error importing file:0316
Error importing file:0518
Error importing file:080331
Error importing file:080422
Error importing file:080426


In [82]:
#for all the files with the error we have the same issue, in the first row there are less elements
#we can just ingore the first row when importing and it should work
failed2 = []
for i in failed:
    path = f"{i}/0.txt"
    try:
        df = pd.read_csv(path, sep = '\t', skiprows = 1, header= None)
    except:
        print(f"Error importing file:{i}")
        failed2.append(i)
        continue
    df_list.append(df)

In [83]:
#verify the we imported everything
print(f"we imported {len(df_list)} datasets out of the {len(folders)} we had originally")

we imported 84 datasets out of the 84 we had originally


In [84]:
data = pd.concat(df_list)

# Data Exploration

In [85]:
data.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,LKh7zAJ4nwo,TheReceptionist,653.0,Entertainment,424.0,13021.0,4.34,1305.0,744.0,DjdA-5oKYFQ,...,vmdPOOd6cxI,KRHfMQqSHpk,pIMpORZthYw,1tUDzOp10pk,heqocRij5P0,_XIuvoH6rUg,LGVU5DsezE0,uO2kj6_D8B4,xiDqywcDQRM,uX81lMev6_o
1,7D0Mf4Kn4Xk,periurban,583.0,Music,201.0,6508.0,4.19,687.0,312.0,e2k0h6tPvGc,...,EzKwOYLh-S0,eUIfRyrqwp8,AK8Wtfwe-1k,Eq4hGkIqBGw,N1lkLaLJHlc,-uIffs-DHkM,zpTorUhCd8Y,AvSK0qPw7EU,WX5KLMqY4bM,VKFqqoeMdjw


In [86]:
# column names
col_names = ["id", "channel", "age", "category", "length", "views", "rate", "ratings","comments"]
for i in range(1,21):
    col_names.append(f"rec_{i}")

In [87]:
data.columns = col_names
data.head()

,id,channel,age,category,length,views,rate,ratings,comments,rec_1,...,rec_11,rec_12,rec_13,rec_14,rec_15,rec_16,rec_17,rec_18,rec_19,rec_20
0,LKh7zAJ4nwo,TheReceptionist,653.0,Entertainment,424.0,13021.0,4.34,1305.0,744.0,DjdA-5oKYFQ,...,vmdPOOd6cxI,KRHfMQqSHpk,pIMpORZthYw,1tUDzOp10pk,heqocRij5P0,_XIuvoH6rUg,LGVU5DsezE0,uO2kj6_D8B4,xiDqywcDQRM,uX81lMev6_o
1,7D0Mf4Kn4Xk,periurban,583.0,Music,201.0,6508.0,4.19,687.0,312.0,e2k0h6tPvGc,...,EzKwOYLh-S0,eUIfRyrqwp8,AK8Wtfwe-1k,Eq4hGkIqBGw,N1lkLaLJHlc,-uIffs-DHkM,zpTorUhCd8Y,AvSK0qPw7EU,WX5KLMqY4bM,VKFqqoeMdjw
2,n1cEq1C8oqQ,Pipistrello,525.0,Comedy,125.0,1687.0,4.01,363.0,141.0,eprHhmurMHg,...,yLup8wjbSIo,lbf4d1pZI9c,uRQYan_-CTQ,gnpvEvuiFoQ,F2_5KOnSsfI,DINu35v3eMU,9uSiyn7t_0o,YfShxdbAJS8,ssdfqTwZXY0,z5wDjq8o60c
3,OHkEzL4Unck,ichannel,638.0,Comedy,299.0,8043.0,4.40,518.0,371.0,eyUSTmEUQRg,...,LQUV_XGzHmA,8OmL_BJRLRw,qeCFW97-fOA,DVNwUKAuB3I,FMuWYExDEJk,rE7TuuXkk4E,bWicrzq2ApQ,jh6EpXnMb18,9JhU2jE02gg,nfBfC8bif1Y
4,-boOvAGNKUc,mrpitifulband,639.0,Music,287.0,7548.0,4.48,606.0,386.0,fmUwUURgsX0,...,eV2SdBITv8k,cIO6nFDnNs4,Bd7nAtOEA3U,RZo5MisSTWo,geiABCqmQ84,MG1Xv99426g,7wj8-HkZ0XQ,JsdCu9T47iY,OUeN4DhCIFw,sf-Ym_pFP6U


In [90]:
len(data.id.unique())

17156

In [89]:
len(data)

17365

In [ ]:
#the data we crawled at diff points in time, so the same video may appear more than once with diff recommendations
#what we do about that?


In [91]:
#store the different edges as a list of tuples
edges = []
for index, row in data.iterrows():
    for i in range(1,21):
        #checkif there is actually a video in the entry
        if row[f"rec_{i}"]== row[f"rec_{i}"]:
            edges.append((row['id'], row[f"rec_{i}"]))
        else:
            #print the nan entry
            print(row['id'], row[f"rec_{i}"])

SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
SDNkMu8ZT68 nan
K4u3KNo2bs8 nan
K4u3KNo2bs8 nan
K4u3KNo2bs8 nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
Gm6XszMrw9Y nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
YjeAkUKPp-w nan
A0eZ9qkUvJU nan
u2pW7PSyZhw nan
u2pW7PSyZhw nan
u2pW7PSyZhw nan
u2pW7PSyZhw nan
u2pW7PSy

6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
6RJgSLCQVvc nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
_OrViN45vOo nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
xGlwGqDvqFc nan
WK3Gv1v4V_M nan
WK3Gv1v4V_M nan
WK3Gv1v4V_M nan
sNCPbJxZnd4 nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNepYk nan
6q-pEGNe

hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
hf-Kn8nFbeA nan
0YSzKZW5uis nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
s7zcCaHTxVs nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
K1gVpLskdD8 nan
W3oC2TZqilI nan
W3oC2TZq

MHkIez1Xq1c nan
MHkIez1Xq1c nan
2XXud9SfkYg nan
2XXud9SfkYg nan
2XXud9SfkYg nan
2XXud9SfkYg nan
2XXud9SfkYg nan
2XXud9SfkYg nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
z9xSSYBWxTw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
BBNu1AErWAw nan
CZrbNSXtBDc nan
CZrbNSXtBDc nan
CZrbNSXtBDc nan
CZrbNSXtBDc nan
CZrbNSXtBDc nan
CZrbNSXtBDc nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNzQBE nan
AJWcbaNz

w5Jaip-zLcQ nan
w5Jaip-zLcQ nan
w5Jaip-zLcQ nan
w5Jaip-zLcQ nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
a2RR5wsLyCM nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
5jO2yB116h0 nan
RZGgBqFHObI nan
RZGgBqFHObI nan
RZGgBqFHObI nan
RZGgBqFHObI nan
RZGgBqFHObI nan
RZGgBqFHObI nan
RZGgBqFHObI nan
RZGgBqFHObI nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1kbo nan
qezuPxE1

...

vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
vY7R4Ro0WSI nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
sSnQHkdYwiU nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
SFHQtb9w30Y nan
XedAehdt8FU nan
XedAehdt8FU nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
1_ypahWCNoc nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84CQI nan
jNK3mg84

jrReN9PJXLI nan
jrReN9PJXLI nan
al91lTz9n6A nan
al91lTz9n6A nan
al91lTz9n6A nan
al91lTz9n6A nan
al91lTz9n6A nan
al91lTz9n6A nan
cRjnzvdly-s nan
cRjnzvdly-s nan
cRjnzvdly-s nan
cRjnzvdly-s nan
cRjnzvdly-s nan
cRjnzvdly-s nan
gVDHL1JWEKo nan
gVDHL1JWEKo nan
gVDHL1JWEKo nan
gVDHL1JWEKo nan
gVDHL1JWEKo nan
gVDHL1JWEKo nan
asDzDC6IEIo nan
asDzDC6IEIo nan
asDzDC6IEIo nan
asDzDC6IEIo nan
asDzDC6IEIo nan
asDzDC6IEIo nan
oKX69i3Yp9k nan
oKX69i3Yp9k nan
oKX69i3Yp9k nan
oKX69i3Yp9k nan
oKX69i3Yp9k nan
oKX69i3Yp9k nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
YWapziQ2mAk nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J4iU nan
MSS4SH1J

EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
EcJ30JzMVQs nan
Qh-D2rbC4k8 nan
Qh-D2rbC4k8 nan
Qh-D2rbC4k8 nan
Qh-D2rbC4k8 nan
Qh-D2rbC4k8 nan
Qh-D2rbC4k8 nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
YLnyBa7jSnc nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQWlAg nan
KppvvMQW

3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
3VUX8r-uqag nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
VHlP4nBJJ94 nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
kJKWl8GTNvQ nan
aalAVhw5p60 nan
aalAVhw5p60 nan
aalAVhw5

F72JuuAwFYU nan
F72JuuAwFYU nan
lEMVSz0gto0 nan
lEMVSz0gto0 nan
lEMVSz0gto0 nan
lEMVSz0gto0 nan
lEMVSz0gto0 nan
lEMVSz0gto0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
CX1sEzf3Kr0 nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
Er7eBgkFVPQ nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNLdgI nan
d4cV-LNL

In [93]:
len(edges)

308942

In [95]:
edges[:20]

[('LKh7zAJ4nwo', 'DjdA-5oKYFQ'),
 ('LKh7zAJ4nwo', 'NxTDlnOuybo'),
 ('LKh7zAJ4nwo', 'c-8VuICzXtU'),
 ('LKh7zAJ4nwo', 'DH56yrIO5nI'),
 ('LKh7zAJ4nwo', 'W1Uo5DQTtzc'),
 ('LKh7zAJ4nwo', 'E-3zXq_r4w0'),
 ('LKh7zAJ4nwo', '1TCeoRPg5dE'),
 ('LKh7zAJ4nwo', 'yAr26YhuYNY'),
 ('LKh7zAJ4nwo', '2ZgXx72XmoE'),
 ('LKh7zAJ4nwo', '-7ClGo-YgZ0'),
 ('LKh7zAJ4nwo', 'vmdPOOd6cxI'),
 ('LKh7zAJ4nwo', 'KRHfMQqSHpk'),
 ('LKh7zAJ4nwo', 'pIMpORZthYw'),
 ('LKh7zAJ4nwo', '1tUDzOp10pk'),
 ('LKh7zAJ4nwo', 'heqocRij5P0'),
 ('LKh7zAJ4nwo', '_XIuvoH6rUg'),
 ('LKh7zAJ4nwo', 'LGVU5DsezE0'),
 ('LKh7zAJ4nwo', 'uO2kj6_D8B4'),
 ('LKh7zAJ4nwo', 'xiDqywcDQRM'),
 ('LKh7zAJ4nwo', 'uX81lMev6_o')]

# Save all the datasets

In [6]:
#save initial dataset just for safety
data.to_csv("youtube_raw.csv",in)

SyntaxError: invalid syntax (729330706.py, line 2)

In [9]:
path = "/Users/Cate/Desktop/DSBA/M2/MLNS/project/youtube_raw.csv"
raw_data = pd.read_csv(path, sep = '\t', header = None)

In [10]:
raw_data.head()

,0
0,"id,channel,age,category,length,views,rate,rati..."
1,"LKh7zAJ4nwo,TheReceptionist,653.0,Entertainmen..."
2,"7D0Mf4Kn4Xk,periurban,583.0,Music,201.0,6508.0..."
3,"n1cEq1C8oqQ,Pipistrello,525.0,Comedy,125.0,168..."
4,"OHkEzL4Unck,ichannel,638.0,Comedy,299.0,8043.0..."


In [108]:
#convert edge list in dataframe
df_edges = pd.DataFrame(edges, columns=["from", "to"])
df_edges.head(2)

,from,to
0,LKh7zAJ4nwo,DjdA-5oKYFQ
1,LKh7zAJ4nwo,NxTDlnOuybo


In [113]:
df_edges.to_csv("edges.edgelist", sep = "\t", index = False, header=False)

In [122]:
nodes = data.iloc[:,:9]
print(len(nodes))
nodes.head(2)


17365


,id,channel,age,category,length,views,rate,ratings,comments
0,LKh7zAJ4nwo,TheReceptionist,653.0,Entertainment,424.0,13021.0,4.34,1305.0,744.0
1,7D0Mf4Kn4Xk,periurban,583.0,Music,201.0,6508.0,4.19,687.0,312.0


In [131]:
#select just unique rows
nodes = nodes.drop_duplicates("id")

In [ ]:
nodes.

# Create network

In [5]:
G = nx.Graph()
G.add_edges_from(edges)

NameError: name 'edges' is not defined

In [4]:
len(list(G.nodes))

NameError: name 'G' is not defined